In [ ]:
%tensorflow_version 1.x
!pip show tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git

In [ ]:
cd '/content/Mask_RCNN'

In [ ]:
!pip3 install -r requirements.txt

In [ ]:
!python3 setup.py install

In [ ]:
!pip show mask-rcnn

In [ ]:
cd

In [ ]:
# example of extracting bounding boxes from an annotation file
from xml.etree import ElementTree

# function to extract bounding boxes from an annotation file
def extract_boxes(filename):
	# load and parse the file
	tree = ElementTree.parse(filename)
	# get the root of the document
	root = tree.getroot()
	# extract each bounding box
	boxes = list()
	for box in root.findall('.//bndbox'):
		xmin = int(box.find('xmin').text)
		ymin = int(box.find('ymin').text)
		xmax = int(box.find('xmax').text)
		ymax = int(box.find('ymax').text)
		coors = [xmin, ymin, xmax, ymax]
		boxes.append(coors)
	# extract image dimensions
	width = int(root.find('.//size/width').text)
	height = int(root.find('.//size/height').text)
	return boxes, width, height

# extract details form annotation file
boxes, w, h = extract_boxes('/content/drive/My Drive/face mask detection data/annotations/maksssksksss0.xml')
# summarize extracted details
print(boxes, w, h)

In [ ]:
cd '/content/Mask_RCNN'

In [ ]:
# split into train and test set
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset

# class that defines and loads the faceMask dataset
class faceMaskDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define one class
		self.add_class("dataset", 1, "without_mask")
		self.add_class("dataset", 2, "with_mask")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annotations/'
		# find all images
		#image_id = 0
		#if not is_train :
		#	image_id = 682
		if is_train:
			for image_id in range(0,682):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1
		else:
			for image_id in range(682,853):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		for box in root.findall('.//object'): #Change required
			name = box.find('name').text #Change required
			xmin = int(box.find('xmin').text)
			ymin = int(box.find('ymin').text)
			xmax = int(box.find('xmax').text)
			ymax = int(box.find('ymax').text)
			coors = [xmin, ymin, xmax, ymax, name] #Change required
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	# load the masks for an image
	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'without_mask'): #Change required #change this to your .XML file
				masks[row_s:row_e, col_s:col_e, i] = 1 #Change required #assign number to your class_id
				class_ids.append(self.class_names.index('without_mask')) #Change required
			else:
				masks[row_s:row_e, col_s:col_e, i] = 2 #Change required
				class_ids.append(self.class_names.index('with_mask')) #Change required

		return masks, asarray(class_ids, dtype='int32')

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

# train set
train_set = faceMaskDataset()
train_set.load_dataset('../drive/My Drive/face mask detection data', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# test/val set
test_set = faceMaskDataset()
test_set.load_dataset('../drive/My Drive/face mask detection data', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

In [ ]:
# plot one photograph and mask
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from matplotlib import pyplot

# class that defines and loads the faceMask dataset
class faceMaskDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define one class
		self.add_class("dataset", 1, "without_mask")
		self.add_class("dataset", 2, "with_mask")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annotations/'
		# find all images
		#image_id = 0
		#if not is_train :
		#	image_id = 682
		if is_train:
			for image_id in range(0,682):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1
		else:
			for image_id in range(682,853):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		for box in root.findall('.//object'): #Change required
			name = box.find('name').text #Change required
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			coors = [xmin, ymin, xmax, ymax, name] #Change required
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	# load the masks for an image
	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'without_mask'): #Change required #change this to your .XML file
				masks[row_s:row_e, col_s:col_e, i] = 1 #Change required #assign number to your class_id
				class_ids.append(self.class_names.index('without_mask')) #Change required
			else:
				masks[row_s:row_e, col_s:col_e, i] = 2 #Change required
				class_ids.append(self.class_names.index('with_mask')) #Change required

		return masks, asarray(class_ids, dtype='int32')

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']





# train set
train_set = faceMaskDataset()
train_set.load_dataset('../drive/My Drive/face mask detection data', is_train=True)
train_set.prepare()
# load an image
image_id = 0
image = train_set.load_image(image_id)
print(image.shape)
# load image mask
mask, class_ids = train_set.load_mask(image_id)
print(mask.shape)
# plot image
pyplot.imshow(image)
# plot mask
pyplot.imshow(mask[:, :, 0], cmap='gray', alpha=0.5)
pyplot.show()

In [ ]:
# plot first few images
for i in range(9):
	# define subplot
	pyplot.subplot(330 + 1 + i)
	# plot raw pixel data
	image = train_set.load_image(i)
	pyplot.imshow(image)
	# plot all masks
	mask, _ = train_set.load_mask(i)
	for j in range(mask.shape[2]):
		pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
# show the figure
pyplot.show()

In [ ]:
# display image with masks and bounding boxes
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes

# class that defines and loads the faceMask dataset
class faceMaskDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define one class
		self.add_class("dataset", 1, "without_mask")
		self.add_class("dataset", 2, "with_mask")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annotations/'
		# find all images
		#image_id = 0
		#if not is_train :
		#	image_id = 682
		if is_train:
			for image_id in range(0,682):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1
		else:
			for image_id in range(682,853):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		for box in root.findall('.//object'): #Change required
			name = box.find('name').text #Change required
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			coors = [xmin, ymin, xmax, ymax, name] #Change required
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	# load the masks for an image
	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'without_mask'): #Change required #change this to your .XML file
				masks[row_s:row_e, col_s:col_e, i] = 1 #Change required #assign number to your class_id
				class_ids.append(self.class_names.index('without_mask')) #Change required
			else:
				masks[row_s:row_e, col_s:col_e, i] = 2 #Change required
				class_ids.append(self.class_names.index('with_mask')) #Change required

		return masks, asarray(class_ids, dtype='int32')

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

# train set
train_set = faceMaskDataset()
train_set.load_dataset('../drive/My Drive/face mask detection data', is_train=True)
train_set.prepare()
# define image id
image_id = 0
# load the image
image = train_set.load_image(image_id)
# load the masks and the class ids
mask, class_ids = train_set.load_mask(image_id)
# extract bounding boxes from the masks
bbox = extract_bboxes(mask)
# display image with masks and bounding boxes
display_instances(image, bbox, mask, class_ids, train_set.class_names)

In [ ]:
cd '/content'

In [ ]:
# fit a mask rcnn on the faceMask dataset
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN

# class that defines and loads the faceMask dataset
class faceMaskDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define one class
		self.add_class("dataset", 1, "without_mask")
		self.add_class("dataset", 2, "with_mask")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annotations/'
		# find all images
		#image_id = 0
		#if not is_train :
		#	image_id = 682
		if is_train:
			for image_id in range(0,682):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1
		else:
			for image_id in range(682,853):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		for box in root.findall('.//object'): #Change required
			name = box.find('name').text #Change required
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			coors = [xmin, ymin, xmax, ymax, name] #Change required
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	# load the masks for an image
	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'without_mask'): #Change required #change this to your .XML file
				masks[row_s:row_e, col_s:col_e, i] = 1 #Change required #assign number to your class_id
				class_ids.append(self.class_names.index('without_mask')) #Change required
			else:
				masks[row_s:row_e, col_s:col_e, i] = 2 #Change required
				class_ids.append(self.class_names.index('with_mask')) #Change required

		return masks, asarray(class_ids, dtype='int32')

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

# define a configuration for the model
class faceMaskConfig(Config):
	# define the name of the configuration
	NAME = "faceMask_cfg"
	# number of classes (background + faceMask)
	NUM_CLASSES = 1 + 2
	# number of training steps per epoch
	STEPS_PER_EPOCH = 682

# prepare train set
train_set = faceMaskDataset()
train_set.load_dataset('drive/My Drive/face mask detection data', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = faceMaskDataset()
test_set.load_dataset('drive/My Drive/face mask detection data', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# prepare config
config = faceMaskConfig()
config.display()
# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
# load weights (mscoco) and exclude the output layers
model.load_weights('drive/My Drive/face mask detection data/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
# train weights (output layers or 'heads')
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=10, layers='heads')

In [ ]:
# evaluate the mask rcnn model on the faceMask dataset
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

# class that defines and loads the faceMask dataset
class faceMaskDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define one class
		self.add_class("dataset", 1, "without_mask")
		self.add_class("dataset", 2, "with_mask")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annotations/'
		# find all images
		#image_id = 0
		#if not is_train :
		#	image_id = 682
		if is_train:
			for image_id in range(0,682):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1
		else:
			for image_id in range(682,853):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		for box in root.findall('.//object'): #Change required
			name = box.find('name').text #Change required
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			coors = [xmin, ymin, xmax, ymax, name] #Change required
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	# load the masks for an image
	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'without_mask'): #Change required #change this to your .XML file
				masks[row_s:row_e, col_s:col_e, i] = 1 #Change required #assign number to your class_id
				class_ids.append(self.class_names.index('without_mask')) #Change required
			else:
				masks[row_s:row_e, col_s:col_e, i] = 2 #Change required
				class_ids.append(self.class_names.index('with_mask')) #Change required

		return masks, asarray(class_ids, dtype='int32')

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "faceMask_cfg"
	# number of classes (background + faceMask)
	NUM_CLASSES = 1 + 2
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		APs.append(AP)
	# calculate the mean AP across all images
	mAP = mean(APs)
	return mAP

# load the train dataset
train_set = faceMaskDataset()
train_set.load_dataset('drive/My Drive/face mask detection data', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# load the test dataset
test_set = faceMaskDataset()
test_set.load_dataset('drive/My Drive/face mask detection data', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# load model weights
model.load_weights('drive/My Drive/face mask detection data/mask_rcnn_faceMask_cfg_0010.h5', by_name=True)
# evaluate model on training dataset
train_mAP = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP)
# evaluate model on test dataset
test_mAP = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP)

In [ ]:
cd /content/

In [ ]:
# detect faceMasks in photos with mask rcnn model
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.model import mold_image
from mrcnn.utils import Dataset

# class that defines and loads the faceMask dataset
class faceMaskDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define one class
		self.add_class("dataset", 1, "without_mask")
		self.add_class("dataset", 2, "with_mask")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annotations/'
		# find all images
		#image_id = 0
		#if not is_train :
		#	image_id = 682
		if is_train:
			for image_id in range(0,682):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1
		else:
			for image_id in range(682,853):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		for box in root.findall('.//object'): #Change required
			name = box.find('name').text #Change required
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			coors = [xmin, ymin, xmax, ymax, name] #Change required
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	# load the masks for an image
	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'without_mask'): #Change required #change this to your .XML file
				masks[row_s:row_e, col_s:col_e, i] = 1 #Change required #assign number to your class_id
				class_ids.append(self.class_names.index('without_mask')) #Change required
			else:
				masks[row_s:row_e, col_s:col_e, i] = 2 #Change required
				class_ids.append(self.class_names.index('with_mask')) #Change required

		return masks, asarray(class_ids, dtype='int32')

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "faceMask_cfg"
	# number of classes (background + faceMask)
	NUM_CLASSES = 1 + 2
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

# plot a number of photos with ground truth and predictions
def plot_actual_vs_predicted(dataset, model, cfg, n_images=1):
	# load image and mask
	for i in range(n_images):
		# load the image and mask
		image = dataset.load_image(i)
		mask, _ = dataset.load_mask(i)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)[0]
		# define subplot
		pyplot.subplot(n_images, 2, i*2+1)
		# plot raw pixel data
		pyplot.imshow(image)
		pyplot.title('Actual')
		# plot masks
		for j in range(mask.shape[2]):
			pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
		# get the context for drawing boxes
		pyplot.subplot(n_images, 2, i*2+2)
		# plot raw pixel data
		pyplot.imshow(image)
		pyplot.title('Predicted')
		ax = pyplot.gca()
		# plot each box
		for box in yhat['rois']:
			# get coordinates
			y1, x1, y2, x2 = box
			# calculate width and height of the box
			width, height = x2 - x1, y2 - y1
			# create the shape
			rect = Rectangle((x1, y1), width, height, fill=False, color='red')
			# draw the box
			ax.add_patch(rect)
	# show the figure
	pyplot.show()

# load the train dataset
train_set = faceMaskDataset()
train_set.load_dataset('drive/My Drive/face mask detection data', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# load the test dataset
test_set = faceMaskDataset()
test_set.load_dataset('drive/My Drive/face mask detection data', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# load model weights
model_path = 'drive/My Drive/face mask detection data/mask_rcnn_faceMask_cfg_0010.h5'
model.load_weights(model_path, by_name=True)
# plot predictions for train dataset
plot_actual_vs_predicted(train_set, model, cfg)
# plot predictions for test dataset
plot_actual_vs_predicted(test_set, model, cfg)

In [ ]:
from mrcnn import visualize
# detect faceMasks in photos with mask rcnn model
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.model import mold_image
from mrcnn.utils import Dataset
import skimage.io

# class that defines and loads the faceMask dataset
class faceMaskDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define one class
		self.add_class("dataset", 1, "without_mask")
		self.add_class("dataset", 2, "with_mask")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annotations/'
		# find all images
		#image_id = 0
		#if not is_train :
		#	image_id = 682
		if is_train:
			for image_id in range(0,682):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1
		else:
			for image_id in range(682,853):

				img_path = images_dir + "maksssksksss"+ str(image_id) + '.png'
				ann_path = annotations_dir +"maksssksksss"+ str(image_id) + '.xml'
				# add to dataset
				self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])
				image_id+=1

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		for box in root.findall('.//object'): #Change required
			name = box.find('name').text #Change required
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			coors = [xmin, ymin, xmax, ymax, name] #Change required
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	# load the masks for an image
	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'without_mask'): #Change required #change this to your .XML file
				masks[row_s:row_e, col_s:col_e, i] = 1 #Change required #assign number to your class_id
				class_ids.append(self.class_names.index('without_mask')) #Change required
			else:
				masks[row_s:row_e, col_s:col_e, i] = 2 #Change required
				class_ids.append(self.class_names.index('with_mask')) #Change required

		return masks, asarray(class_ids, dtype='int32')

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "kangaroo_cfg"
	# number of classes (background + kangaroo)
	NUM_CLASSES = 1 + 2
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

# plot a number of photos with ground truth and predictions
def plot_actual_vs_predicted(dataset, model, cfg, n_images=5):
	# load image and mask
	for i in range(n_images):
		# load the image and mask
		image = dataset.load_image(i)
		#mask, _ = dataset.load_mask(i)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)[0]
		# define subplot
		class_names = ['BG', 'without_mask', 'with_mask']
		visualize.display_instances(image, yhat['rois'], yhat['masks'], yhat['class_ids'], class_names, yhat['scores'])
		print(yhat['class_ids'])  
		pyplot.subplot(n_images, 2, i*2+1)
		# plot raw pixel data
		pyplot.imshow(image)
		pyplot.title('Actual')
		# plot masks
		for j in range(mask.shape[2]):
			pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
		# get the context for drawing boxes
		pyplot.subplot(n_images, 2, i*2+2)
		# plot raw pixel data
		pyplot.imshow(image)
		pyplot.title('Predicted')
		ax = pyplot.gca()
		# plot each box
		for box in yhat['rois']:
			# get coordinates
			y1, x1, y2, x2 = box
			# calculate width and height of the box
			width, height = x2 - x1, y2 - y1
			# create the shape
			rect = Rectangle((x1, y1), width, height, fill=False, color='red')
			# draw the box
			ax.add_patch(rect)
	# show the figure
	pyplot.show()

# load the train dataset
train_set = KangarooDataset()
train_set.load_dataset('drive/My Drive/face mask detection data', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# load the test dataset
test_set = KangarooDataset()
test_set.load_dataset('drive/My Drive/face mask detection data', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# load model weights
model_path = 'drive/My Drive/face mask detection data/mask_rcnn_faceMask_cfg_0010.h5'
model.load_weights(model_path, by_name=True)
# plot predictions for train dataset
#plot_actual_vs_predicted(train_set, model, cfg)
# plot predictions for test dataset
#plot_actual_vs_predicted(test_set, model, cfg)
image = skimage.io.imread('new.jpg')

results = model.detect([image], verbose=1)
class_names = ['BG', 'without_mask', 'with_mask']
# Visualize results

r = results[0]

if 2 in r['class_ids']:
	print("<-------------------With mask-------------------->>")
else:
	print("<-------------------Without mask-------------------->>")
 
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])